<a href="https://colab.research.google.com/github/felicia2602/trial/blob/main/Copy_of_testing_association_rule.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ***Case Study* 03 - *Association Rule* (*Market Basket Analysis*) - Kelompok G - *Data Mining***

1. Annisa Fairuz Zahira (2006571015)
2. Daniel Rivaldo Haryanto (2006534890)
3. Nicholas Sky Salvatio (2006532701)
4. Inez Aurellia Rosyana (2006571204)
5. Olivia Felica Utomo (2006464114)
6. Owen Susanto (2006570933)
7. Widyadari Dhiya Parahita (2006572913)

In [ ]:
pip install mlxtend

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
pip install apyori

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for apyori: filename=apyori-1.1.2-py3-none-any.whl size=5974 sha256=b855d7060676b7fa95bf131997f034fdc3309ef3ae083fd782f94a791154e9b1
  Stored in directory: /root/.cache/pip/wheels/cb/f6/e1/57973c631d27efd1a2f375bd6a83b2a616c4021f24aab84080
Successfully built apyori


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from apyori import apriori
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
import datetime

In [ ]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from google.colab import drive
from oauth2client.client import GoogleCredentials

In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
link = 'https://drive.google.com/open?id=15OFTU5ia9-uV9VbnoPMpUXJArFOsufYH'
fluff, id = link.split('=')
print(id)
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('testing.xlsx')  
df = pd.read_excel('testing.xlsx')

15OFTU5ia9-uV9VbnoPMpUXJArFOsufYH


In [ ]:
df

,ID Transaksi,Tanggal,Konsumen,ID Produk,Nama Produk,Merek,Jumlah
0,1,2015-07-01 00:00:00,Mr. A,1,Roti,Sri Roti,2
1,1,2015-07-01 00:00:00,Mr. A,2,Air Minum,Aquos,1
2,1,2015-07-01 00:00:00,Mr. A,11,Mie Instan,Indomi,2
3,1,2015-07-01 00:00:00,Mr. A,12,Mie Instan,Mi Sedap,1
4,2,2015-07-01 00:00:00,Mr. B,2,Air Minum,Aquos,1
5,2,2015-07-01 00:00:00,Mr. B,6,Snack,Chiki,2
6,2,2015-07-01 00:00:00,Mr. B,3,Roti,Prambanan,1
7,2,2015-07-01 00:00:00,Mr. B,5,Shampoo,Clean,1
8,3,2015-07-02 00:00:00,Mr. C,4,Sabun,Lifboy,1
9,3,2015-07-02 00:00:00,Mr. C,5,Shampoo,Clean,1


In [ ]:
df.dtypes

ID Transaksi     int64
Tanggal         object
Konsumen        object
ID Produk        int64
Nama Produk     object
Merek           object
Jumlah           int64
dtype: object

# ***Pre-Processing***

In [ ]:
df['Tanggal'] = pd.to_datetime(df['Tanggal'])

In [ ]:
df2 = df.drop(columns=['Konsumen','ID Produk','Merek'])

In [ ]:
df2

,ID Transaksi,Tanggal,Nama Produk,Jumlah
0,1,2015-07-01,Roti,2
1,1,2015-07-01,Air Minum,1
2,1,2015-07-01,Mie Instan,2
3,1,2015-07-01,Mie Instan,1
4,2,2015-07-01,Air Minum,1
5,2,2015-07-01,Snack,2
6,2,2015-07-01,Roti,1
7,2,2015-07-01,Shampoo,1
8,3,2015-07-02,Sabun,1
9,3,2015-07-02,Shampoo,1


# ***Data Transformation***

In [ ]:
basket = df.groupby(['ID Transaksi','Nama Produk'])['Jumlah'].count().unstack().reset_index().fillna(0).set_index('ID Transaksi')
basket.head()

Nama Produk,Air Minum,Mie Instan,Rokok,Roti,Sabun,Shampoo,Snack
ID Transaksi,,,,,,,
1,1.0,2.0,0.0,1.0,0.0,0.0,0.0
2,1.0,0.0,0.0,1.0,0.0,1.0,1.0
3,0.0,0.0,1.0,0.0,1.0,1.0,0.0


In [ ]:
def encode_units(x):
  if x <= 0:
    return 0
  if x >= 1:
    return 1

basket_sets = basket.applymap(encode_units)
basket_sets.head()

Nama Produk,Air Minum,Mie Instan,Rokok,Roti,Sabun,Shampoo,Snack
ID Transaksi,,,,,,,
1,1,1,0,1,0,0,0
2,1,0,0,1,0,1,1
3,0,0,1,0,1,1,0


# ***Data Mining***

In [ ]:
frequent_itemsets = apriori(basket_sets, min_support=0.1, use_colnames=True)
frequent_itemsets

,support,itemsets
0,0.666667,(Air Minum)
1,0.333333,(Mie Instan)
2,0.333333,(Rokok)
3,0.666667,(Roti)
4,0.333333,(Sabun)
5,0.666667,(Shampoo)
6,0.333333,(Snack)
7,0.333333,"(Mie Instan, Air Minum)"
8,0.666667,"(Roti, Air Minum)"
9,0.333333,"(Air Minum, Shampoo)"


In [ ]:
rules1 = association_rules(frequent_itemsets, metric='lift', min_threshold=1)
rules1.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Mie Instan),(Air Minum),0.333333,0.666667,0.333333,1.0,1.5,0.111111,inf
1,(Air Minum),(Mie Instan),0.666667,0.333333,0.333333,0.5,1.5,0.111111,1.333333
2,(Roti),(Air Minum),0.666667,0.666667,0.666667,1.0,1.5,0.222222,inf
3,(Air Minum),(Roti),0.666667,0.666667,0.666667,1.0,1.5,0.222222,inf
4,(Snack),(Air Minum),0.333333,0.666667,0.333333,1.0,1.5,0.111111,inf


In [ ]:
results1 = rules1[(rules1['lift'] >= 1) &
                  (rules1['confidence'] >= 0.8)]

apr_result = results1.sort_values(by='lift', ascending=False)
apr_result.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
36,"(Air Minum, Shampoo)",(Snack),0.333333,0.333333,0.333333,1.0,3.0,0.222222,inf
37,(Snack),"(Air Minum, Shampoo)",0.333333,0.333333,0.333333,1.0,3.0,0.222222,inf
61,"(Air Minum, Shampoo)","(Roti, Snack)",0.333333,0.333333,0.333333,1.0,3.0,0.222222,inf
60,"(Roti, Shampoo)","(Air Minum, Snack)",0.333333,0.333333,0.333333,1.0,3.0,0.222222,inf
57,"(Air Minum, Snack)","(Roti, Shampoo)",0.333333,0.333333,0.333333,1.0,3.0,0.222222,inf


# **Interpretasi**

In [ ]:
apr_result['consequents'].value_counts()

(Air Minum)                   8
(Roti)                        8
(Shampoo)                     7
(Snack)                       3
(Air Minum, Roti)             3
(Air Minum, Shampoo)          2
(Roti, Shampoo)               2
(Rokok)                       2
(Sabun)                       2
(Roti, Snack)                 1
(Air Minum, Snack)            1
(Rokok, Shampoo)              1
(Sabun, Shampoo)              1
(Air Minum, Roti, Shampoo)    1
Name: consequents, dtype: int64